# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Признаки:
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

# 0. Подготовка инструментов

In [1]:
pip install -U numpy

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: numpy in /home/jovyan/.local/lib/python3.7/site-packages (1.19.2)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U pandas

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: pandas in /home/jovyan/.local/lib/python3.7/site-packages (1.1.2)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scikit-learn in /home/jovyan/.local/lib/python3.7/site-packages (0.23.2)
Note: you may need to restart the kernel to use updated packages.


# 1. Загрузка и подготовка данных

Загрузите и подготовьте данные. Поясните порядок действий.

Данные без дефектов, поэтому этап предобработки можем пропустить.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df0 = pd.read_csv('/datasets/geo_data_0.csv')
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df2 = pd.read_csv('/datasets/geo_data_2.csv')
print(df0, end='\n')
print(df1, end='\n')
print(df2, end='\n')

          id        f0        f1        f2     product
0      txEyH  0.705745 -0.497823  1.221170  105.280062
1      2acmU  1.334711 -0.340164  4.365080   73.037750
2      409Wp  1.022732  0.151990  1.419926   85.265647
3      iJLyR -0.032172  0.139033  2.978566  168.620776
4      Xdl7t  1.988431  0.155413  4.751769  154.036647
...      ...       ...       ...       ...         ...
99995  DLsed  0.971957  0.370953  6.075346  110.744026
99996  QKivN  1.392429 -0.382606  1.273912  122.346843
99997  3rnvd  1.029585  0.018787 -1.348308   64.375443
99998  7kl59  0.998163 -0.528582  1.583869   74.040764
99999  1CWhH  1.764754 -0.266417  5.722849  149.633246

[100000 rows x 5 columns]
          id         f0         f1        f2     product
0      kBEdx -15.001348  -8.276000 -0.005876    3.179103
1      62mP7  14.272088  -3.475083  0.999183   26.953261
2      vyE1P   6.263187  -5.948386  5.001160  134.766305
3      KcrkZ -13.081196 -11.506057  4.999415  137.945408
4      AHL4O  12.702195  -8.

In [5]:
print(df0.info(), end='\n')
print(df1.info(), end='\n')
print(df2.info(), end='\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

# 2. Обучение и проверка модели

Обучите и проверьте модель для каждого региона:
- 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
- 2.2. Обучите модель и сделайте предсказания на валидационной выборке.
- 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
- 2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
- 2.5. Проанализируйте результаты.

In [6]:
def train_valid(df):
    
    df_work = df.drop(['id'], axis=1)
    features = df_work.drop(['product'], axis=1)
    target = df_work['product']

    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12)
    
    model = LinearRegression(n_jobs=-1)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    
    df_pred = pd.DataFrame()
    df_pred = features_valid.merge(target_valid, left_index=True, right_index=True)
    df_pred = df_pred.merge(df['id'], left_index=True, right_index=True)
    df_pred = df_pred[['id', 'f0', 'f1', 'f2', 'product']]
    df_pred['predict'] = predict_valid
    
    return df_pred
    
def rmse(df):
    
    rmse = mean_squared_error(df['product'], df['predict'], squared=False)
    
    print(f'RMSE - {rmse :.2f}')
    print(f'Средний запас предсказанный - {df.predict.mean() :.2f}')

In [7]:
df0_train = train_valid(df0)
rmse(df0_train)

RMSE - 37.79
Средний запас предсказанный - 92.13


In [8]:
df1_train = train_valid(df1)
rmse(df1_train)

RMSE - 0.89
Средний запас предсказанный - 69.04


In [9]:
df2_train = train_valid(df2)
rmse(df2_train)

RMSE - 39.80
Средний запас предсказанный - 95.05


RMSE показывает отклонение в исходных единицах, в данном случае - в бареллях. Т.е. на такое число может быть отклонение от среднего значения. Можно сказать, что запас в df1 во всех скважинах примерно одинаков. Возможно, в df0 и df2 имеются выбросы.

# 3. Подготовка к расчёту прибыли

Подготовьтесь к расчёту прибыли:
- 3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.
- 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
- 3.3. Напишите выводы по этапу подготовки расчёта прибыли.

- При разведке региона исследуют 500 точек, из которых выбирают 200 лучших для расчёта прибыли.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- Один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [10]:
# Можем перевести в тыс. рублей бюджет и доход.
BUDGET = 10000000
INCOME = 450

# прибыль считают по 200 точек
WELL_REGION = 200

# на разработку одной скважины
ONE_WELL = BUDGET / WELL_REGION

PRODUCT_COUNT = ONE_WELL / INCOME
print(f'безубыточная разработка одной скважины, ед. продукта - {PRODUCT_COUNT :.2f}')

# средний запас по всему датасету
def product_mean(df):
    product_mean = df['product'].mean()
    print(f'средний запас в регионе - {product_mean :.2f}')

безубыточная разработка одной скважины, ед. продукта - 111.11


In [11]:
product_mean(df0)

средний запас в регионе - 92.50


In [12]:
product_mean(df1)

средний запас в регионе - 68.83


In [13]:
product_mean(df2)

средний запас в регионе - 95.00


### Вывод

Считаем прибыль по 200 точкам, а среднее по 500 для снижения рисков. Пока все 3 региона выглядят перспективно.

# 4. Расчёт прибыли и рисков 

Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- 4.1. Выберите скважины с максимальными значениями предсказаний. Количество скважин зависит от бюджета компании и стоимости разработки одной скважины.
- 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
- 4.3. Рассчитайте прибыль для полученного объёма сырья.

In [14]:
def product_predict(df, rand):
    state = np.random.RandomState(rand)
    df_tmp = df.sample(n=500, replace=True, random_state=state)
    
    df_tmp = df_tmp.sort_values('predict', ascending=False).head(200)

    # Находим прибыль по каждой.
    df_tmp['income_product'] = (df_tmp['product'] - PRODUCT_COUNT) * INCOME
    
    return df_tmp.income_product.sum()

In [15]:
print(f'Прибыль по региону, тыс. руб. - {product_predict(df0_train, 12) :.2f}')

Прибыль по региону, тыс. руб. - 219999.08


In [16]:
print(f'Прибыль по региону, тыс. руб. - {product_predict(df1_train, 12) :.2f}')

Прибыль по региону, тыс. руб. - 213710.20


In [17]:
print(f'Прибыль по региону, тыс. руб. - {product_predict(df2_train, 12) :.2f}')

Прибыль по региону, тыс. руб. - 512307.62


# 5. Посчитаем риски и прибыль для каждого региона

Посчитайте риски и прибыль для каждого региона:
- 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- 5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

In [18]:
def income_bootstrap(df):

    values_inc = []
    for i in range(1000):
        values_inc.append(product_predict(df, np.random.randint(0, 1000)))

    values_inc = pd.Series(values_inc)
    lower = values_inc.quantile(0.025)
    high = values_inc.quantile(0.975)
    
    print(f'Прибыли меньше нуля - {values_inc[values_inc < 0].count() / values_inc.count() :.2%}')
    print(f'Средняя прибыль - {values_inc.mean()}')
    print(f'2.5%-квантиль - {lower}')
    print(f'97.5%-квантиль - {high}')

In [19]:
income_bootstrap(df0_train)

Прибыли меньше нуля - 4.40%
Средняя прибыль - 467911.70495395205
2.5%-квантиль - -71584.44768597808
97.5%-квантиль - 1015415.3624080548


In [20]:
income_bootstrap(df1_train)

Прибыли меньше нуля - 1.30%
Средняя прибыль - 475601.79072609183
2.5%-квантиль - 53161.54751688691
97.5%-квантиль - 851616.1406721886


In [21]:
income_bootstrap(df2_train)

Прибыли меньше нуля - 7.90%
Средняя прибыль - 354895.5930327972
2.5%-квантиль - -159945.68537562495
97.5%-квантиль - 905827.7072435699


### Вывод

Согласно предсказанию, следует выбрать регион координаты скважин которого занесены в датафрейм df1. RMSE на нем показывала минимальную ошибку. И также bootstrap показала лучшие результаты, фактически риск выхода в убыток ниже 2,5%. Таким образом, можно заключить, что регион df1 наиболее привлекательный с инвестиционной точки зрения.